In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import json

In [12]:
def generate_average_df(algorithms, data_sets, results_path, output_path): 
    output = {}
    for algorithm in algorithms: 
        output[algorithm] = {}
        for data_set in data_sets: 
            print(algorithm, data_set)
            try:
                results = pd.read_csv(os.path.join(results_path, algorithm, data_set,'results_table.csv'), header=None)
                results = results[4].values
                output[algorithm][data_set] = str(round(np.mean(results), 3)) + " (" + str(round(np.std(results),3)) +")" 
            except: 
                output[algorithm][data_set] = "NA"
    output_df = pd.DataFrame.from_dict(output, orient="index")
    output_df.to_latex(os.path.join(output_path, 'average_f1_table.txt'))

In [3]:
def generate_histograms(algorithms, data_sets, results_path, output_path, data_path):
    output = {}
    for algorithm in algorithms: 
        output[algorithm] = {}
        for data_set in data_sets: 
            files = glob.glob(os.path.join(results_path, algorithm, data_set, '*.json'))
            if not files:
                output[algorithm][data_set] = "N/A"
                continue
            try:
                ARL = []
                for file in files: 
                    with open(file) as f:
                        result = json.load(f)
                    ts_name = result['data_name']
                    if result['score'] is None: continue
                    detected = np.array(result['cp'])[:, 0]
                    labels = pd.read_csv(os.path.join(data_path, data_set,"{}_labels.csv".format(ts_name)), header=None)
                    labels = labels.values[:,0]
                    selected = np.zeros(len(detected))
                    print(len(detected), len(selected))
                    for cp in labels:
                        dist = (detected-cp).astype(float)
                        dist[dist < 0] = np.inf
                        min_ = np.min(np.abs(dist) + selected *10000)
                        index = np.argmin(np.abs(dist) + selected *10000)
                        if min_ < 200:
                            ARL.append(dist[index])
                            selected[index] = 1
                print(algorithm, data_set, len(ARL))
#                 output[algorithm][data_set] = str(round(np.mean(np.abs(ARL)), 3)) + " (" + str(round(np.std(np.abs(ARL)), 3)) +"), " + str(round(np.median(np.abs(ARL)), 3)) 
                output[algorithm][data_set] =  str(round(np.median(np.abs(ARL)), 3))+ f'({str(len(ARL))})' 
#                 
#                 fig = plt.figure(figsize=(6,4))
#                 plt.hist(ARL, bins = 20, color = 'g', alpha=0.7)
#                 plt.title('{} ({})'.format(data_set, algorithm))
#                 fig.savefig('{}/{}_{}'.format(output_path, data_set, algorithm), dpi = 400)
            except: 
                output[algorithm][data_set] = "N/A"
    output_df = pd.DataFrame.from_dict(output)
    output_df.to_latex(os.path.join(output_path, 'arl_table.txt'))
    
 

In [15]:
data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data')
results_path = os.path.join(os.path.dirname(os.getcwd()), 'test')
output_path = os.path.join(os.path.dirname(os.getcwd()), 'processing')
algorithms = ['mssa_mw', 'binseg', 'microsoft_ssa', 'bocpdms', 'klcpd']
data_sets = ['struct', 'mean', 'energy', 'yahoo', 'hasc']
generate_average_df(algorithms, data_sets, results_path, output_path)
# generate_histograms(algorithms, data_sets, results_path, output_path, data_path)

mssa_mw struct
mssa_mw mean
mssa_mw energy
mssa_mw yahoo
mssa_mw hasc
binseg struct
binseg mean
binseg energy
binseg yahoo
binseg hasc
microsoft_ssa struct
microsoft_ssa mean
microsoft_ssa energy
microsoft_ssa yahoo
microsoft_ssa hasc
bocpdms struct
bocpdms mean
bocpdms energy
bocpdms yahoo
bocpdms hasc
klcpd struct
klcpd mean
klcpd energy
klcpd yahoo
klcpd hasc


In [5]:
files = glob.glob(os.path.join(results_path, algorithms[3], data_sets[0], '*.json'))
files

['/Users/abdullah/Dropbox (MIT)/GitHub/cpd_benchmark/results/test/density_ratio/struct/struct11_417a135f-db7d-4d7f-a2e9-ae7c9e00d43c.json',
 '/Users/abdullah/Dropbox (MIT)/GitHub/cpd_benchmark/results/test/density_ratio/struct/struct16_c9600a9c-7025-4fd3-8a0d-eecf85b5d939.json',
 '/Users/abdullah/Dropbox (MIT)/GitHub/cpd_benchmark/results/test/density_ratio/struct/struct13_4d8bfc23-6f51-4681-9a04-e7ad6b67363a.json',
 '/Users/abdullah/Dropbox (MIT)/GitHub/cpd_benchmark/results/test/density_ratio/struct/struct18_f7564bc9-c254-40b4-9ed3-0ada22aee0ee.json',
 '/Users/abdullah/Dropbox (MIT)/GitHub/cpd_benchmark/results/test/density_ratio/struct/struct2_73d3b79d-eccd-445a-92ee-d5470b74a505.json',
 '/Users/abdullah/Dropbox (MIT)/GitHub/cpd_benchmark/results/test/density_ratio/struct/struct12_ae71ba31-9881-421a-9b90-a3333f8c37b2.json',
 '/Users/abdullah/Dropbox (MIT)/GitHub/cpd_benchmark/results/test/density_ratio/struct/struct7_404f140c-83af-4ea8-9c0a-d77e00a93c4f.json',
 '/Users/abdullah/Drop

In [20]:
for file in files: 
    with open(file) as f:
        result = json.load(f)
    ts_name = result['data_name']
    print(np.array(result['cp']))
    np.array(result['cp'])[:, 0]

[[    0   299]
 [  300   479]
 [  480   826]
 [  827  1942]
 [ 1943  2015]
 [ 2016  4832]
 [ 4833  5999]
 [ 6000  6773]
 [ 6774  8827]
 [ 8828  9240]
 [ 9241 10751]
 [10752 11767]]
[    0 11856]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed